CLONING THE REQUIRED FILES
1. Midi Neural Processor to 

In [ ]:
!git clone https://github.com/asigalov61/midi-neural-processor
!git clone https://github.com/asigalov61/MusicTransformer-Pytorch

INSTALLING THE REQUIRED LIBRARIES

In [ ]:
!pip install requirements.txt

Importing the Required Modules

In [ ]:
import numpy as np
import pickle
import os
import sys
import math
import random
import pypianoroll
from pypianoroll import Multitrack, Track
import matplotlib
import matplotlib.pyplot as plt
import mir_eval.display
import librosa
import librosa.display
import pretty_midi
from midi2audio import FluidSynth
from IPython.display import display, Javascript, HTML, Audio

Baseline Model:
    <br>1. Preprocess the Maestro dataset
    <br>2. Training the Music Transformer on the Maestro Dataset for 150 epochs

In [ ]:
!mkdir ./MusicTransformer-Pytorch/dataset/
%cd ./dataset
!wget 'https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip'
!unzip maestro-v2.0.0-midi.zip


Preprocess the Maestro Dataset

In [ ]:
%cd ./MusicTransformer-Pytorch/
!python3 preprocess_midi.py './dataset/maestro-v2.0.0'

Training the model

In [ ]:
batch_size = 4
number_of_training_epochs = 150
maximum_output_MIDI_sequence = 1280

!python3 train.py -output_dir rpr --rpr -batch_size=$batch_size -epochs=$number_of_training_epochs -max_sequence=$maximum_output_MIDI_sequence #-n_layers -num_heads -d_model -dim_feedforward

Evaluate the model
<br>Best Resulting Accuracy

In [ ]:
!python3 evaluate.py -model_weights rpr/results/best_acc_weights.pickle --rpr

Best Loss Model

In [ ]:
!python3 evaluate.py -model_weights rpr/results/best_loss_weights.pickle --rpr

Preprocess Custom MIDI Dataset

In [ ]:
# %cd MusicTransformer-Pytorch
from processor import encode_midi

import os
import random

%cd './dataset/e_piano/custom_midis'

custom_MIDI_DataSet_dir = './dataset/e_piano/custom_midis'

train_dir = './dataset/e_piano/train' # split_type = 0
test_dir = './dataset/e_piano/test' # split_type = 1  
val_dir = './dataset/e_piano/val' # split_type = 2

total_count = 0
train_count = 0
val_count   = 0
test_count  = 0

f_ext = '.pickle'
fileList = os.listdir(custom_MIDI_DataSet_dir)
print(fileList)
for file in fileList:
     # we gonna split by a random selection for now
    
    split = random.randint(1, 2)
    if (split == 0):
         o_file = os.path.join(train_dir, file+f_ext)
         train_count += 1

    elif (split == 2):
         o_file0 = os.path.join(train_dir, file+f_ext)
         train_count += 1
         o_file = os.path.join(val_dir, file+f_ext)
         val_count += 1

    elif (split == 1):
         o_file0 = os.path.join(train_dir, file+f_ext)
         train_count += 1
         o_file = os.path.join(test_dir, file+f_ext)
         test_count += 1
    try:
      print(o_file0)
      prepped = encode_midi(file)
      o_stream = open(o_file0, "wb")
      pickle.dump(prepped, o_stream)
      o_stream.close()

      prepped = encode_midi(file)
      o_stream = open(o_file, "wb")
      pickle.dump(prepped, o_stream)
      o_stream.close()
   
      print(file)
      print(o_file)
      print('Coverted!')  
    except KeyboardInterrupt: 
      raise   
    except:
      print('Bad file. Skipping...')

print('Done')
print("Num Train:", train_count)
print("Num Val:", val_count)
print("Num Test:", test_count)
print("Total Count:", train_count)



Training the Model on our Custom Dataset
<br> Using the checkpoint of 150 epochs of Base Model
<br> Training for 50 more epochs on our dataset

In [ ]:
batch_size = 4
number_of_training_epochs = 200
maximum_output_MIDI_sequence = 512
saved_checkpoint_full_path = "./rpr/weights/epoch_0150.pickle" 
continue_epoch_number = 150
!python3 train.py -output_dir rpr --rpr -batch_size=$batch_size -epochs=$number_of_training_epochs -max_sequence=$maximum_output_MIDI_sequence -continue_weights $saved_checkpoint_full_path -continue_epoch $continue_epoch_number

Evaluating our models

In [ ]:
!python3 evaluate.py -model_weights rpr/results/best_acc_weights.pickle --rpr